In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Model
#print(tf.keras.__version__)
#print(tf.__version__)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math as mt
import random as rng

from sklearn import preprocessing

def getColumns(size):
    columns = []
    for i in range(size):
        columns.append(i)
    return columns

directory = "./CMAPSSData/train_FD001.txt"
turbo = pd.read_csv(directory, header=None, sep=' ')
turbo = turbo.drop([26, 27], axis=1)

#for i in turbo.columns:
#    column = turbo[i]
#    print(i)
#    print(column.describe())
    
#print(turbo.values)

# Drop the input parameters to get just the sensors
turboSens = turbo.drop([0,1], axis=1)
turboSens.columns = getColumns(turboSens.shape[1])

# Get the cycles and useful sensors. 
turboUsefulSens = turboSens

min_max_scaler = preprocessing.MinMaxScaler()
norm_sens = min_max_scaler.fit_transform(turboUsefulSens)

turboCycles = turbo[1]
#print(turboCycles)

deteriorate = []
maxCycle = float(turboCycles.max())

currentCount = maxCycle
for i in range(turbo.shape[0] - 1, -1, -1):
    currentRow = turboCycles.loc[i]
    deteriorate.insert(0, currentCount)
    if currentRow == 1:
        currentCount = maxCycle
    else:
        currentCount -= 1

        
deteriorate = pd.DataFrame(deteriorate)

deteriorateSwap = []
for i in deteriorate[0]:
    deteriorateSwap.append(maxCycle - i)
deteriorate = pd.DataFrame(deteriorateSwap)


norm_sens = pd.DataFrame(norm_sens)
deteriorate = deteriorate.join(norm_sens, lsuffix='0', rsuffix='1')
deteriorate.columns = getColumns(deteriorate.shape[1])



Using TensorFlow backend.


In [2]:
array = deteriorate.to_numpy()

arrayCopy = array.tolist()

simulations = []

convCount = 10
convDiv = int(convCount/2)

count = len(array)

while(count > 0):
    cycles = []
    element = [-1,-1]
    while(element[0]!=0):
        element = arrayCopy.pop(0)
        cycles.append(element)
        count-=1
    simulations.append(cycles)
    
#for sim in simulations:
#    lenth = len(sim)
#    remainder = lenth%convCount
#    for i in range(remainder):
#        sim.pop(0)
    
#simsRecomb = []
#for sim in simulations:
#    for i in sim:
#        simsRecomb.append(i)

#print(len(simsRecomb))

In [3]:

tens = []
tensCount = []

for sim in simulations:
    for i in range(convDiv, len(sim) - convDiv):
        ten = []
        total = 0
        for j in range(-convDiv,convDiv):
            index = i + j
            element = sim[index].copy()
            total+= element.pop(0)
            ten.append(element)
        total/=convCount
        tensCount.append(total)
        tens.append(ten)

    
print(len(tens))
print(len(tensCount))

19631
19631


In [4]:
simsReorder = []
simsCycleReorder = []

length = len(tens)
for i in range(length):
    index = rng.randint(0, len(tens)-1)
    simsReorder.append(tens.pop(index))
    simsCycleReorder.append(tensCount.pop(index))
    
    
    
simsTrain = np.array(simsReorder[:15704])
targetTrain = np.array(simsCycleReorder[:15704])

simsTest = np.array(simsReorder[15704:])
targetTest = np.array(simsCycleReorder[15704:])



width = (turboUsefulSens.shape[1])
print(width)

24


In [5]:
visible = layers.Input(shape=(convCount, width,))

cnn1 = layers.Conv1D(150, 3, activation='relu')(visible)
cnn2 = layers.Conv1D(300, 3, activation='relu')(cnn1)
pool = layers.MaxPooling1D(6)(cnn2)
drop = layers.Dropout(0.25)(pool)
flat = layers.Flatten()(pool)

lstm1 = layers.LSTM(150, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(visible)
lstm2 = layers.LSTM(300, dropout=0.1, recurrent_dropout=0.1)(lstm1)

combo = layers.merge.concatenate([flat,lstm2])
dense = layers.Dense(100, activation='relu')(combo)
out = layers.Dense(1)(dense)

model = Model(inputs=visible, outputs=out)




print(model.summary())


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10, 24)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 8, 150)       10950       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 6, 300)       135300      conv1d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 1, 300)       0           conv1d_2[0][0]                   
____________________________________________________________________________________________

In [6]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='mse', 
              metrics=['mae', 'mse'])

In [7]:
model.fit(simsTrain, targetTrain, epochs=750, verbose=1, validation_data=(simsTest,targetTest))

Train on 15704 samples, validate on 3927 samples
Epoch 1/750
15704/15704 [==============================] - 23s 1ms/step - loss: 2263.6504 - mae: 35.0192 - mse: 2263.6497 - val_loss: 1640.7981 - val_mae: 29.4764 - val_mse: 1640.7982
Epoch 2/750
15704/15704 [==============================] - 22s 1ms/step - loss: 1736.7208 - mae: 30.0306 - mse: 1736.7201 - val_loss: 1599.3762 - val_mae: 28.5164 - val_mse: 1599.3760
Epoch 3/750
15704/15704 [==============================] - 24s 2ms/step - loss: 1680.6161 - mae: 29.2834 - mse: 1680.6166 - val_loss: 1551.8131 - val_mae: 28.3170 - val_mse: 1551.8131
Epoch 4/750
15704/15704 [==============================] - 22s 1ms/step - loss: 1645.6455 - mae: 28.6733 - mse: 1645.6453 - val_loss: 1535.8922 - val_mae: 28.3367 - val_mse: 1535.8923
Epoch 5/750
15704/15704 [==============================] - 22s 1ms/step - loss: 1610.8964 - mae: 28.3509 - mse: 1610.8970 - val_loss: 1493.7266 - val_mae: 27.3982 - val_mse: 1493.7268
Epoch 6/750
15704/15704 [======

In [9]:
model.evaluate(simsTest,targetTest)

3927/3927 [==============================] - 2s 401us/step


[35.82863289240756, 4.098214626312256, 35.82863998413086]

In [10]:
for i in range(100):
    test = np.array([simsTest[i]])
    print(model.predict(test, verbose=1))
    print(targetTest[i])



1/1 [==============================] - 0s 129ms/step
[[53.631084]]
62.5
1/1 [==============================] - 0s 2ms/step
[[103.068344]]
99.5
1/1 [==============================] - 0s 3ms/step
[[147.82161]]
156.5
1/1 [==============================] - 0s 2ms/step
[[97.04354]]
81.5
1/1 [==============================] - 0s 3ms/step
[[132.71924]]
135.5
1/1 [==============================] - 0s 3ms/step
[[275.43393]]
268.5
1/1 [==============================] - 0s 2ms/step
[[101.08215]]
99.5
1/1 [==============================] - 0s 2ms/step
[[59.951275]]
61.5
1/1 [==============================] - 0s 3ms/step
[[116.662544]]
126.5
1/1 [==============================] - 0s 3ms/step
[[151.58888]]
149.5
1/1 [==============================] - 0s 3ms/step
[[97.1438]]
93.5
1/1 [==============================] - 0s 3ms/step
[[106.10655]]
108.5
1/1 [==============================] - 0s 3ms/step
[[126.30682]]
127.5
1/1 [==============================] - 0s 3ms/step
[[168.36891]]
174.5
1/1 [======

In [11]:
model_json = model.to_json()
with open("./model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Saved model to disk


In [13]:
np.save("./valData.npy", simsTest)
np.save("./valTarget.npy", targetTest)